In [15]:
import xapian

dbpath = 'xapIndex_SenID_Text'
datapath = 'wiki-pages-text/'

TOP_RESULTS_LIMIT = 15
def search(dbpath, querystring, offset=0, pagesize=10):
    
    database = xapian.Database(dbpath)
    enquire = xapian.Enquire(database)
    query_string = querystring

    qp = xapian.QueryParser()
    stemmer = xapian.Stem("english")
    qp.set_stemmer(stemmer)
    qp.set_database(database)
    
    qp.set_stemming_strategy(xapian.QueryParser.STEM_SOME)
    query = qp.parse_query(query_string)
    #print "Parsed query is: %s" % str(query)

    # Find the top results for the query.
    enquire.set_query(query)
    matches = enquire.get_mset(0, TOP_RESULTS_LIMIT)

    # Display the results.
    #print(%i results found." % matches.get_matches_estimated()
    #print "Results 1-%i:" % matches.size()
    
    text_dictionary = {}
    for m in matches:
#         print('RANK:', m.rank + 1)
#         print('PERCENTAGE MATCH:', m.percent)
#         print('DOC ID:', m.docid)
#         print('DOC TXT:', m.document.get_data())
        decoded_text = m.document.get_data().decode('utf-8')
        text_dictionary[(decoded_text.split(",,,")[1],decoded_text.split(",,,")[2])] =decoded_text.split(",,,")[0]
                                                                                  

    return text_dictionary


query = "When in rome do as romans do"
matches = search(dbpath, query)

In [37]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

query_list = [query]*TOP_RESULTS_LIMIT
feature_db = pd.DataFrame({"claim": query_list,"evidence":tuple(matches.keys())})
#nltk.download('punkt')
import nltk
stemmer = nltk.stem.PorterStemmer()
def jaccard_coefficient(claim,evidence_list):
    final_claims = []
    final_evidences = []
    claim_tokens = nltk.word_tokenize(claim[0])
    for claims in claim_tokens:
        stemmed_lowercased_claim = stemmer.stem(claims.lower())
        final_claims.append(stemmed_lowercased_claim)
    #print(final_claims)
    
    #print(set(claim[0].split()))
    #print(list(set(claim)))
    #print(("".join(evidence_list).split()))
    evidence_tokens = nltk.word_tokenize("".join(evidence_list))
    for evidence in evidence_tokens:
        stemmed_lowercased_evidence = stemmer.stem(evidence.lower())
        final_evidences.append(stemmed_lowercased_evidence)
    #print(final_evidences)
    #print(["".join(evidence_list).split()])
    intersection = len(list(set(final_claims).intersection(set(final_evidences))))
    #print(intersection)
    union = (len(claim) + len(evidence_list)) - intersection
    lower = min(len(final_claims),len(final_evidences))
    jacc_LCS = []
    jacc_LCS.append((1 - float(intersection / union)))
    jacc_LCS.append(float(intersection/lower))
    return jacc_LCS


# def Longest_Common_Subsequence(claim,evidence_list):
#     final_claims = []
#     final_evidences = []
#     claim_tokens = nltk.word_tokenize(claim[0])
#     for claims in claim_tokens:
#         stemmed_lowercased_claim = stemmer.stem(claims.lower())
#         final_claims.append(stemmed_lowercased_claim)
#     evidence_tokens = nltk.word_tokenize("".join(evidence_list))
#     for evidence in evidence_tokens:
#         stemmed_lowercased_evidence = stemmer.stem(evidence.lower())
#         final_evidences.append(stemmed_lowercased_evidence)
#     intersection = len(list(set(final_claims).intersection(set(final_evidences))))
#     lower = min(len(final_claims),len(final_evidences))
#     return float(intersection/lower)

def noun_count(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc:
        if token.pos_ == "PROPN" or token.pos_ == "NOUN":
            count +=1
    return count

In [38]:

#def create_features(feature_db):
feature_db['jaccard_similarity'] = [(jaccard_coefficient(feature_db['claim'],evidence))[0] for evidence in matches.values()]
feature_db['compare_length'] = [(len(evidence) - len(feature_db['claim']))/len(evidence) for evidence in matches.values()]
feature_db['LCS'] = [(jaccard_coefficient(feature_db['claim'],evidence))[1] for evidence in matches.values()]
feature_db['Noun_ratio'] = [noun_count(feature_db['claim'][0])/noun_count(evidence) for evidence in matches.values()]
#return feature_db
feature_db

,claim,evidence,jaccard_similarity,compare_length,LCS,Noun_ratio
0,When in rome do as romans do,"(When_in_Rome, 3)",0.929412,0.802632,0.857143,0.666667
1,When in rome do as romans do,"(When_in_Rome,_do_as_the_Romans_do, 6)",0.942308,0.842105,0.857143,0.500000
2,When in rome do as romans do,"(When_in_Rome,_do_as_the_Romans_do, 0)",0.979522,0.947183,0.857143,0.166667
3,When in rome do as romans do,"(When_in_Rome_Do_as_The_Vandals, 0)",0.983287,0.957143,0.857143,0.086957
4,When in rome do as romans do,"(Greta_Rana, 11)",0.977860,0.942748,0.857143,0.181818
5,When in rome do as romans do,"(Peace_thru_Vandalism, 3)",0.945055,0.814815,0.714286,0.285714
6,When in rome do as romans do,"(Anapodoton, 5)",0.972603,0.928571,0.857143,0.285714
7,When in rome do as romans do,"(When_in_Rome,_do_as_the_Romans_do, 5)",0.974359,0.896552,0.571429,0.285714
8,When in rome do as romans do,"(Roman_Party, 1)",0.977941,0.942966,0.857143,0.200000
9,When in rome do as romans do,"(No_Apologies_-LRB-The_Eyeliners_album-RRB-, 1)",0.973684,0.893617,0.571429,0.200000


In [30]:
# query_list = [query]*TOP_RESULTS_LIMIT
# dataset = pd.DataFrame({"claim": query_list,"evidence":tuple(matches.keys())})

#create_features(dataset)

In [24]:
import numpy as np
data = pd.read_json('test-unlabelled.json',orient="index")
data.head(10)

#create_features(data)

,claim
110000,Raven-Symoné is an Anglican.
73397,Temple of the Dog celebrated the 37th annivers...
16079,Solanum contains plants with ornamental flowers.
114789,A Good Day to Die Hard is part of the Library ...
126871,The Icelandic Coast Guard is also known as Gae...
226458,Chadwick Boseman portrayed a film character.
132649,Ghost is a film.
51263,James Brolin is an orthodontist.
51262,Superhuman abilities may result from antelope ...
51260,Robinson Crusoe on Mars was a Greek tragedy.


In [22]:
len(data['claim'])

14997

In [44]:
datasubset = data.iloc[0:20]
datasubset.loc[np.repeat(datasubset.index.values, TOP_RESULTS_LIMIT)]

,claim
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.
110000,Raven-Symoné is an Anglican.


In [45]:
#matches is iteratively changed
from tqdm import tqdm
match_dictionary = {}
with tqdm(total=len(datasubset['claim'])) as pbar:
    for index in range(0,len(datasubset['claim'])):
        matches = search(dbpath, datasubset['claim'].iloc[index])
        match_dictionary.update(matches)
    #[matches = search(dbpath, data['claim'].iloc[index]) for index in range(len(data['claim']))]
    pbar.update(1)
    

datasubset['evidence'] = tuple(match_dictionary.keys())

  5%|▌         | 1/20 [00:19<06:07, 19.36s/it]


ValueError: Length of values does not match length of index

In [ ]:
data.head()